In [12]:
import time
import numpy as np
np.random.seed(42)

In [13]:
def identity_matrix(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

def zeros_matrix(rows, cols):
    return [[0 for _ in range(cols)] for _ in range(rows)]

def transpose(M):
    return [[M[j][i] for j in range(len(M))] for i in range(len(M[0]))]

def dot(u, v):
    return sum([u[i] * v[i] for i in range(len(u))])

def norm(v):
    return sum([v[i] * v[i] for i in range(len(v))]) ** 0.5

def matmul(A, B):
    result = zeros_matrix(len(A), len(B[0]))
    for i in range(len(A)):
        for j in range(len(B[0])):
            result[i][j] = sum([A[i][k] * B[k][j] for k in range(len(B))])
    return result

def scalar_multiply(scalar, v):
    return [scalar * vi for vi in v]

def vector_subtract(u, v):
    return [u[i] - v[i] for i in range(len(u))]

def gram_schmidt_qr(A):
    n = len(A)
    m = len(A[0])
    Q = []
    R = zeros_matrix(m, m)
    
    for j in range(m):
        a_j = [A[i][j] for i in range(n)]
        q_j = a_j[:]
        
        for i in range(j):
            q_i = [Q[row][i] for row in range(n)]
            R[i][j] = dot(q_i, a_j)
            proj = scalar_multiply(R[i][j], q_i)
            q_j = vector_subtract(q_j, proj)
        
        R[j][j] = norm(q_j)
        q_j = [val / R[j][j] for val in q_j]
        
        if j == 0:
            Q = [[val] for val in q_j]
        else:
            for row in range(n):
                Q[row].append(q_j[row])
                
    return Q, R

def upper_tri_inverse(R, tol=1e-6): 
    n = len(R)
    R_inv = zeros_matrix(n, n)

    for i in range(n - 1, -1, -1):
        if abs(R[i][i]) < tol: 
            R_inv[i][i] = 1 / (R[i][i] if abs(R[i][i]) >= tol else tol)
        else:
            R_inv[i][i] = 1 / R[i][i]
        
        for j in range(i + 1, n):
            total = 0
            for k in range(i + 1, j + 1):
                total += R[i][k] * R_inv[k][j]
            R_inv[i][j] = -total / R[i][i]

    return R_inv

def qr_inverse(A):
    Q, R = gram_schmidt_qr(A)
    R_inv = upper_tri_inverse(R)
    Q_T = transpose(Q)
    return matmul(R_inv, Q_T)



In [14]:
def generate_random_matrix(n):
    return np.random.uniform(1, 10, (n, n)).tolist()
matrix_sizes = [10, 100, 1000]

for size in matrix_sizes:
    A = generate_random_matrix(size)
    start_time = time.time()
    A_inv = qr_inverse(A)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"\nSize {size}x{size}: Execution time = {execution_time:.6f} seconds")
    print(f"First 3 rows of inverse matrix {size}x{size}:")
    for row in A_inv[:3]:
        print(row)


Size 10x10: Execution time = 0.094197 seconds
First 3 rows of inverse matrix 10x10:
[0.08506733866730615, -0.12091183769121415, 0.020822287161011768, -0.04888465169702905, -0.05730204847582418, 0.055701912841062275, -0.03800020827999786, 0.03275471360382812, 0.04968467103265961, 0.0026729954357474785]
[-0.14220523942221935, -0.21976464424711445, -0.22535125670786893, 0.05814973515322995, 0.24720536988421796, 0.17549064624402377, -0.0693153201292869, -0.3842244855102973, 0.31166549493272033, 0.18321426564845977]
[0.12352845165158816, 0.3726841488493581, 0.21086290885171932, -0.06761382896621806, -0.28623095693404366, -0.1196959094882203, 0.04218153482094772, 0.38599069466832414, -0.33855358408662367, -0.2126676660165168]

Size 100x100: Execution time = 0.133874 seconds
First 3 rows of inverse matrix 100x100:
[-0.10303194795628617, 0.10139714915461367, -0.11724192761523286, -0.10339206185911101, 0.03316885069981769, 0.06871078230791723, -0.0907522951683703, -0.35178304215509326, 0.04095